# ML Programmentwurf

> **Algorythmus**: Klassifikation
>
> **Datensatz**: Iris
>
> **Autor**: Oliver Seider


## Dataset Einlesen

Der Datensatz wird mit Pandas eingelesen. Es werden zusätzlich Namen für die Spalten definiert, obwohl diese ab dem nächten Schritt nicht mehr von Relevanz sind.

Anschließend wird der Datensatz mit Hilfe von *numpy* in ein zweidimensionales Array umgewandelt.

In [ ]:
import pandas as pd


headers = ['sepal length', 'sepal width', 'petal length', 'petal width', 'category']
pandas_dataframe = pd.read_csv('iris.data', header=None, names=headers)

array_2d = pandas_dataframe.to_numpy()

# print(pandas_dataframe)
# print(array_2d)
print('There are {0} samples in the dataset'.format(len(array_2d)))

## Umformen der Daten
Das Zweidimensionale Daten-Array wird in einem nächten Schritt in Features und Kategorie aufgeteilt.

In [ ]:
data_array_2d = array_2d[:150,:4]
category_array = array_2d[:150, 4:].reshape([150])

# print(data_array_2d)
# print(category_array)

## Aufteilen der Daten in Lern- und Testdaten
Der Test-Datensatz stellt 25 Prozent der Originaldaten dar. Bei der Aufspaltung wurde ein Zufallskoeffizient von 0 gewählt, was bedeutet dass immer die selbe Aufteilung ausgegeben wird.

In [ ]:
from sklearn.model_selection import train_test_split

reshaped_data_train, reshaped_data_test, target_data_train, target_data_test = train_test_split(data_array_2d, category_array, test_size=0.25, random_state=0)

## Erstellen des Modells
In dem nächsten Schritt werden **KNeighborsClassifier** mit 0-20 *neigbors* erstellt und mit Hilfe des ***cross-validation*** Algorythmus von *sklearn* beweertet, um die beste Anzahl an *neighbours* herauszufinden.

Durch den 'return_estimator=True' Parameter bei der *cross-validation* werden die *k* Modelle, die bei der *cross-validation* erstellt wurden, zurückgegeben.

Erzielt die Anzahl an *neighbours* einen besseren durchschnittlichen Score in der *cross-validation*, wird das Modell geespeichert.

Am Ende wird das beste Modell für den weiteren Verlauf mit den Trainings-Daten 'trainiert'.

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.neighbors import KNeighborsClassifier
import numpy as np

best_model = [None, 0, 0]
scores = []

for n in range(1, 20):
    model = KNeighborsClassifier(n_neighbors=n)
    results = cross_validate(model, reshaped_data_train, target_data_train, cv=5, return_estimator=True)
    average_score = np.mean(results['test_score'])
    if average_score > best_model[1]:
        best_model = [model, average_score, n]
    scores.append(average_score)

# print(best_model)
print(f'The best average score ({best_model[1]}) is archived with {best_model[2]} neighbours.')
best_model = best_model[0].fit(reshaped_data_train, target_data_train)
# print(scores)

## Testen des Modells
Nun werden die Testdaten anhand des besten Modells klassifiziert.

In [ ]:

# Klassen-Angehörigkeit von jedem Datensatatz nach dem Modell
y = best_model.predict(reshaped_data_test)

# print(y)

## Evaluierung des Modells
In diesem letzten Schritt werden die Ergebnisse aus dem Vorherigen Schritt anhand von einem *Classification Report* und einer *Confusion Matrix* evaluiert.

**Classification Report:**
- *Precision*: Gibt an, wie viele der identifizierten Elemente *true positives* sind (true positives / [true positives + false positives]).
- *Recall*: Gibt an, wie viele Elemente, die einer Klasse angehören, dieser auch zugeteilt wurden (true positives / [true positives + false negatives])
- *F1-Score*: Harmonischer Mittelwert aus *Precision* und *Recall*
- *Support*: Anzahl an Datensätzen, die zum Bestimmen der vorherigen Werte verwendet wurden.

**Confusion Matrix:**
Zweidimensionale Matrix, die für jede Klasse und deren Datensätze zeigt, wie häufig das Modell einen Datensatz einer Klasse zugeordnet hat. 
Dabei stellt jede Spalte eine Klasse und dessen Datensätze dar, während die Spalten angeben wie oft eine Datensatz dieser Klasse zugeordnet wurde.

In [ ]:
from sklearn import metrics


print('Classification Report:')
print(metrics.classification_report(target_data_test, y))

print('Confusion Matrix:')
print(metrics.confusion_matrix(target_data_test, y))

Der *Classification Report* zeigt, dass dieses Modell einen durchschnittlichen F1-Score von 0,97 erhält, was über 0,9 liegt und somit als sehr gut klassifiziert werden kann.